## INF367 Mandatory Assignment 2

<p style="text-align:right;"><b>Written by:</b> Tobias Husebø, Lasse Holt, Martin Flo Øfstaas</p>
    <p style="text-align:right;"><i>Due: 8th of November 2024, (12:00)</i></p>
    
---

### Data exploration:


In [ ]:
import qiskit 
from sklearn import datasets
import matplotlib.pyplot as plt

data = datasets.load_iris()
x = data.data
columns = data.feature_names
y = data.target

# Data exploration 
print(x.shape) # 150 samples, 4 features 
print(len(set(y))) # 3 classes

# The range of values for each feature
for column in range(x.shape[1]):
    min_val = x[:, column].min()
    max_val = x[:, column].max()
    print(f"Feature {column}: {min_val} to {max_val}")

# The distribution of the features
plt.figure(figsize=(20, 5))
for column in range(x.shape[1]):
    plt.subplot(1, 4, column+1)
    plt.hist(x[:, column], bins=20)
    plt.title(columns[column])

# Splitting the data 

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets
x_train, x_val_test, y_train, y_val_test = train_test_split(x, y, test_size=0.30, random_state=42, stratify=y)
# Splitting the training set into training and validation sets
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=0.5, random_state=42, stratify=y_val_test)

# Data Encoding

In [ ]:
import numpy as np

#Scaling the data in range 0 to 2pi for angle encoding
def dataScaling(x_values):
    for i in range(x_values.shape[1]):
        x_values[:, i] = 2 * np.pi * (x_values[:, i] - x_values[:, i].min()) / (x_values[:, i].max() - x_values[:, i].min())
    return x_values

# Loss function

In [ ]:
def multiClassCrossEntropy(prediction, label):
    return -(label * np.log(prediction))

# Real amplitudes

In [ ]:
def angleEncoding(feature_vector, n_qubits):
    circuit = qiskit.QuantumCircuit(n_qubits)
    # Input encoding
    for i in range(n_qubits):
        circuit.rx(feature_vector[i], i)
    return circuit

angleEncoding(x[1], 4).draw(output='mpl')


In [ ]:
import random
import qiskit
import numpy as np
def paramterizedCircuit(n_qubits, n_layers):
    circuit = qiskit.QuantumCircuit(n_qubits)
    param = qiskit.circuit.ParameterVector('θ', n_qubits*n_layers)
    paramindex = 0
    for layer in range(n_layers):
        # Paramterized layer
        for i in range(n_qubits):
            circuit.ry(param[paramindex], i)
            paramindex += 1
        # Adding barrier to separate the layers
        circuit.barrier()
        # Entanglement layer
        for i in range(n_qubits-1):
            circuit.cx(i, i+1)
        # Adding barrier to separate the layers
        if layer < n_layers-1:
            circuit.barrier()
    # Measurment 
    circuit.measure_all()
    return circuit, param

circuit, param = paramterizedCircuit(4, 2)
params = np.random.uniform(0, 2*np.pi, len(param))
circuit.assign_parameters({param[i]: params[i] for i in range(len(param))})
circuit.draw(output='mpl')


In [ ]:
class IrisQNN:
    def __init__(self, n_qubits, n_layers):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.n_params = n_qubits * n_layers
        self.params = ParameterVector('θ', self.n_params)
    
    def angle_encoding(self, qc, input_features):
        number_of_qubits = qc.num_qubits
        for qubit in range(number_of_qubits):
            qc.rx(input_features[qubit], qubit)
        qc.barrier()
        return qc

    def add_variational_layer_real_amplitude(self, qc, layer_idx):
        param_offset = layer_idx * self.n_qubits
        for i in range(self.n_qubits):
            qc.ry(self.params[param_offset + i], i)
        
        qc.barrier()
        
        for i in range(self.n_qubits - 1):
            qc.cx(i, i+1)
            
        if layer_idx < self.n_layers - 1:
            qc.barrier()
            
        return qc

    def create_circuit(self, input_features):
        qc = QuantumCircuit(self.n_qubits)
        
        self.angle_encoding(qc, input_features)
        
        for layer in range(self.n_layers):
            self.add_variational_layer_real_amplitude(qc, layer)
           
        return qc
    
    def _get_parameter_count(self):
        return self.n_params

In [ ]:
qnn = IrisQNN(n_qubits=4, n_layers=2)

feature = x_train[0]

qc = qnn.create_circuit(feature)
qc.draw('mpl')